In [2]:
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split

In [3]:
# input = np.load("input.npy")
# output = np.load("output.npy")
# x = np.array([input**2, input]).T
# y = output.reshape((output.shape[0], -1))

# tts = train_test_split(x, y)
# train_x = tts[0][0]
# train_y = tts[2][0]
# test_x = tts[1][0]
# test_y = tts[3][0]

# def gen_x(N, d):
#     arr = np.random.randint(100, size=(N,d))
#     return arr

# def gen_y(x):
#     s = np.sum(x, axis=1)
#     boolean = x[:,0] / s
#     return (boolean >= 0.5).astype(np.float32)

def gen_x(N, d):
    arr = np.random.randint(100, size=(N,d))
    return arr

def gen_y(x):
    return (x[:, 0] >= 50).astype(np.float32)

# 2 Neural Networks
## Problem 2.1 Perception

## Gradient Descent Derivation
### Loss function gradient
$\ell(y_p, y_g) = -y_glog(y_p) - (1-y_g)log(1-y_p)$

$y_p = \sigma(w^Tx + w_0)$

$\nabla_w\ell(y_p, y_g) = \nabla_w[-y_glog(\sigma(w^Tx+w_0)) - (1-y_g)log(1-\sigma(w^Tx+w_0))]$

$= -y_g(\nabla_w\sigma(w^Tx+w_0))(\frac{1}{\sigma(w^Tx+\theta_0)}) - (1-y_g)(\nabla_w\sigma(w^Tx+w_0))(-\frac{1}{1-\sigma(w^Tx+w_0)})$

$\nabla_w\ell = (-\frac{y_g}{y_p} + \frac{1-y_g}{1-y_p})\nabla_wy_p$



### Non-linear Function (sigmoid) gradient

$y_p = \sigma(w^Tx+w_0)$

$\nabla_wy_p = \nabla_w\sigma(w^Tx+w_0)$

$u = w^Tx+w_0 = y_p$

$\sigma(u) = \frac{1}{1+e^{-u}}$

$\nabla_wy_p = \frac{\delta\sigma}{\delta u}\frac{\delta u}{\delta w}$

$\frac{\delta\sigma}{\delta u} = (e^{-u})(\frac{1}{(1+e^{-u})^2}) = \frac{e^{-u}}{(1+e^{-u})^2} = \frac{1}{1+e^{-u}}(\frac{1+e^{-u}}{1+e^{-u}} - \frac{1}{1+e^{-u}}) = \sigma(u)(1-\sigma(u))$

$\frac{\delta u}{\delta w} = x$

$\nabla_wy_p = \sigma(y_p)(1-\sigma(y_p))x$


### Gradient Update Step
Since we have the gradient, we can use newton's method to approach the minimum loss given some weights for each update. Since we want to find the minimum, we subtract the gradient. We also add in a learning rate as a hyperparameter in order to not overshoot or end up in a local minima.

$w_{n+1} = w_n - \alpha\nabla_w\ell$

so we have

$w_{n+1} = w_n - \alpha(-\frac{y_g}{y_p} + \frac{1-y_g}{1-y_p})\nabla_wy_p = w_n - \alpha[(-\frac{y_g}{y_p} + \frac{1-y_g}{1-y_p})][y_p(1-y_p)][x]$

Since we are only using one point per update step, we would perform this step for every single epoch. If it was more than one point, then we can take the average of the gradients so it looks something like this:

$w_{n+1} = w_n - \alpha(\frac{1}{N}\sum_{i=1}^N-\frac{y_{g,i}}{y_{g,i}} + \frac{1-y_{g,i}}{1-y_{p,i}})\nabla_wy_p$

For $N$ number of points and where $y_{g,i}$ represents $y_g$ at point $i$ and $y_{p,i}$ represents $y_p$ at point $i$.

## Problem 2.2 Backpropagation
Let us start by defining our problem to be a $k$ layer neural network with input $x_k$ and output $y_{p,k}$, where $x_k$ is the input at layer $k$ and $y_{p,k}$ is the predicted output at layer $k$. We have labels $y$ and input into the neural net $x$ which correspond to $y_k$ and $x_0$ respectively.b We have weights $w$ and a non-linear function $\sigma$. The first layer is right after inputing $y_p^0 = x_1$. Note that also $x^{i} = y_p^{i-1}$. We have $1, ..., k$ layers.

$y_{p}^k = \sigma^k(\sum_i w^k_ix^{k-1}_i)$

Thus at layer $L$ we have:
$y_{p}^L = \sigma^k(\sum_i w^{L}_ix^{L-1}_i)$

Let's have $u^k = \sum_i w_i^kx^k_i$

The Loss function is defined as:
$\ell(y_g, y_p^{L}) = \ell(y, I(\sum_i w_i^{L}x^{L-1}_i)) = \ell(y, (\sum_i w_i^{L}y_{p,i}^{L-1}))$

The generalized formula of the gradient of the loss function with respect to $w$ is:

$\nabla_w\ell = \frac{\delta \ell}{\delta \sigma} \frac{\delta \sigma}{\delta u}\frac{\delta u}{\delta w}$

Note the gradient is a vector, and we can say that there are $j$ weights so we should denote each weight as $w_j$ giving us

$\nabla_{w_j}\ell = \frac{\delta \ell}{\delta \sigma} \frac{\delta \sigma}{\delta u}\frac{\delta u}{\delta w_j}$

And so then we can further generalize it to the $k$th layer by doing this:

$\nabla_{w_j}\ell^k = \frac{\delta \ell}{\delta \sigma^k} \frac{\delta \sigma^k}{\delta u^k}\frac{\delta u^k}{\delta w^k_j}$

In the proof, we will just solve for an aribtrary weight $w_j$ but note that we have to take its derivative with respect to each weight and that is what $w_j$ represents.

Useful equations we will use in the future:

$\nabla_wy_p = \frac{\delta\sigma}{\delta u}\frac{\delta u}{\delta w}$

$\frac{\delta\sigma}{\delta u} = (e^{-u})(\frac{1}{(1+e^{-u})^2}) = \frac{e^{-u}}{(1+e^{-u})^2} = \frac{1}{1+e^{-u}}(\frac{1+e^{-u}}{1+e^{-u}} - \frac{1}{1+e^{-u}}) = \sigma(u)(1-\sigma(u))$

$\frac{\delta u}{\delta w} = x$

$\nabla_wy_p = \sigma(y_p)(1-\sigma(y_p))x$

Since our last layer is not non-linear as we are interested in regression, we have:

$y_p^L = \sum_i w_i^Lx_i^{L-1}$

and our loss function is $L_2$, so it is given by:

$\ell(y_g, y_p^L) = (y_g - y_p^L)^2$

Now we use our chain rule above and get:

$\nabla_w\ell^L = 2*(y_g - y_p^L) \frac{\delta y_p}{\delta w} = 2*(y_g - \sum_iw_i^Lx_i^{L-1})\frac{\delta y_p}{\delta w_j} = 2*(y_g - \sum_iw_i^Lx_i^{L-1})(-x^{L-1})$

Now for some given layer $k$, we have:

$\frac{\delta \ell}{\delta w^k} = \frac{\delta \ell^k}{\delta u^k}\frac{\delta u^k}{\delta w^k}$

$u^k = \sum_i w_i^k x^{k} = \sum_i w_i^k y_p^{k-1}$

$\frac{\delta u^k}{\delta w^k} = y_p^{k-1}$

$\frac{\delta \ell}{\delta u^k} = \sum_i \frac{\delta \ell}{\delta u_i^{k+1}}\frac{\delta u_i^{k+1}}{\delta u^k}$

$u_i^{k+1} = \sum_s w_{s,i}^{k+1}y_{p,s}^{k} = \sum_s w_{s,i}^{k+1}\sigma(u^{k})$

$\frac{\delta u_i^{k+1}}{\delta u^k} = \sigma'(u^k) w_{ji}^{k+1}$

$\frac{\delta \ell}{\delta u^k} = \sum_i \frac{\delta \ell}{\delta u_i^{k+1}}\sigma'(u^k_j)w_ji^{k+1}$

$ = \sigma'(u^k_j) \sum_i \frac{\delta \ell}{\delta u_i^{k+1}}w_ji^{k+1}$

Now that we have the gradient of the $k$ th layer relative to the $k+1$ layer, we have base case $k+1 = L$ so we can do back propagation. Here are the important equation takeaways.

For layer $L$ as the last layer, the gradient is given by:

$\frac{\delta \ell(w)}{\delta w^L} = 2*(y_g - \sum_i w^Lx^{L-1})(-x^{L-1})$

$\frac{\delta \ell(w)}{\delta u^L} = 2*(y_g - \sum_i w^Lx^{L-1})$

For layer $k$ as the any layer from $1..L-1$, the gradient to update the weights is given by:

$\frac{\delta \ell(w)}{\delta w^k} = \sigma(u^k)(1-\sigma(u^k)) (\sum_i \frac{\delta \ell(w)}{\delta u_i^{k+1}}w_{ji}^{k+1}) y_p^{k-1}$

# Feel free to ignore all code below

In [10]:
# def cross_entropy_loss(w, x, y_g):
#     # y_g is {-1, 1}
#     return -y_g*np.log(w*x) - (1-y_g)*np.log(1-w*x)

def cross_entropy_loss(y_p, y_g):
    # y_g is {-1, 1}
    return -y_g*np.log(y_p) - (1-y_g)*np.log(1-y_p)

def cross_entropy_deriv(w, x, y_g, predict):
    y_p = predict(w, x)
    return -(y_g / y_p) + (1-y_g) / (1-y_p)

def sigmoid(x):
    # assert type(x) == np.ndarray
    denominator = 1 + np.exp(-x)
    return 1 / denominator

def sgn(x):
    return np.sign(x)

def hinge_loss(w, x, y_g, predict):
    y_p = predict(w, x)
    return np.sum(np.maximum(0, 1-y_p*y_g))

def hinge_deriv(w, x, y_g, predict):
    y_p = predict(w, x)
    if y_p*y_g < 1:
        return np.sum(-y_g*x)
    return 0

In [5]:
class Perceptron:
    """Currently only supports a very specific implementation
    """

    def __init__(self, x, y, nlf, ell, dell, loss, lr=0.01, tol=0.01):
        assert (x.shape[0], 1) == (y.shape[0], 1), "Make sure your matrices are (N,d) and (N,1)" # (Class, dim)
        # assert len(x.shape) == 2, "Make sure x is 2D. If not do reshape((N, -1))"
        x = x.reshape((x.shape[0], -1))

        self.nlf_ = nlf # Non linear function (sigmoid)
        self.ell_ = ell
        self.dell_ = dell # Derivative of Loss Function
        self.loss_ = loss
        self.lr_ = lr
        self.tol_ = tol
        self.num_points_ = x.shape[0] # N
        self.dim_ = x.shape[1] + 1 # Add one to dimension for bias, d
        self.w_ = np.random.rand(self.dim_) # Initialize random weights [0, 1)
        self.w_ /= la.norm(self.w_)
        self.x_ = x.copy()# Append a one at beginning for bias
        # np.insert(x.reshape((self.num_points_, -1)), 0, np.ones(self.num_points_), axis=1) 
        self.y_g = y.T.flatten().copy()

    def mini_batch_gradient_descent(self, lr=0.01, batch_size=5, epochs=25):
        error = []
        for _ in range(epochs):
            idx = np.random.randint(self.num_points_, size=batch_size)
            grad = 0
            err = 0
            for i in idx:
                err += self.ell_(self.w_, self.x_[i], self.y_g[i], self.predict)
                grad += self.dell_(self.w_, self.x_[i], self.y_g[i], self.predict)
            grad /= idx.shape[0]
            err /= idx.shape[0]
            error.append((_, err))
            self.w_ = self.w_ - lr * grad
        return self.w_, error
    
    def predict(self, w, x):
        """Performs feed forward with weight summation and non-linear activation
        """
        # assert row.shape[0] == self.dim_, str(row.shape) + " " + str(self.dim_)
        mean = np.average(x)
        std = np.std(x)
        normalized_x = (x - mean) / std
        return self.nlf_(np.sum(w[1:] * normalized_x) + w[0])
        
    def prediction(self, x):
        return self.predict(self.w_, x)

In [6]:
train_x = gen_x(100, 3)
train_y = gen_y(train_x)

In [7]:
# p = Perceptron(train_x, train_y, sigmoid, cross_entropy_loss, cross_entropy_deriv, cross_entropy_loss)
p = Perceptron(train_x, train_y, sgn, hinge_loss, hinge_deriv, cross_entropy_loss)
w, error = p.mini_batch_gradient_descent(batch_size=5, epochs=100)
error

[(0, 0.4),
 (1, 0.8),
 (2, 0.6),
 (3, 0.2),
 (4, 0.8),
 (5, 0.6),
 (6, 0.0),
 (7, 0.8),
 (8, 0.4),
 (9, 0.4),
 (10, 0.6),
 (11, 0.6),
 (12, 0.6),
 (13, 0.4),
 (14, 0.4),
 (15, 0.6),
 (16, 0.2),
 (17, 0.6),
 (18, 0.4),
 (19, 0.2),
 (20, 0.4),
 (21, 0.8),
 (22, 0.6),
 (23, 0.8),
 (24, 0.4),
 (25, 0.6),
 (26, 0.8),
 (27, 0.4),
 (28, 0.2),
 (29, 0.6),
 (30, 0.6),
 (31, 0.8),
 (32, 0.4),
 (33, 0.6),
 (34, 0.4),
 (35, 0.6),
 (36, 0.4),
 (37, 0.0),
 (38, 0.0),
 (39, 0.2),
 (40, 0.4),
 (41, 0.4),
 (42, 0.4),
 (43, 0.4),
 (44, 0.2),
 (45, 0.6),
 (46, 0.6),
 (47, 0.8),
 (48, 0.6),
 (49, 0.8),
 (50, 0.6),
 (51, 0.8),
 (52, 0.6),
 (53, 0.4),
 (54, 0.4),
 (55, 0.8),
 (56, 0.8),
 (57, 0.4),
 (58, 0.6),
 (59, 0.6),
 (60, 0.2),
 (61, 0.4),
 (62, 0.2),
 (63, 0.6),
 (64, 0.4),
 (65, 0.6),
 (66, 0.4),
 (67, 0.6),
 (68, 0.2),
 (69, 0.4),
 (70, 0.4),
 (71, 0.4),
 (72, 0.4),
 (73, 0.6),
 (74, 0.2),
 (75, 0.6),
 (76, 0.4),
 (77, 0.8),
 (78, 0.2),
 (79, 0.6),
 (80, 0.6),
 (81, 0.4),
 (82, 0.8),
 (83, 0.2),
 (

In [8]:
test_x = gen_x(1, 3)
test_y = gen_y(test_x)
test_x[0,0], p.prediction(test_x), test_y

(59, 1.0, array([1.], dtype=float32))

In [9]:
p = Perceptron(train_x, train_y, sigmoid, cross_entropy_deriv, cross_entropy_loss)
y_p = p.feed_forward()
y_p[:5]

TypeError: __init__() missing 1 required positional argument: 'loss'

In [ ]:
cross_entropy_deriv(y_p[:5], train_y[:5].flatten())
# # np.apply_along_axis(cross_entropy_deriv, 0, )
# # train_y[:5].T
# y_p.shape, train_y.shape
idx = np.random.randint(y_p.shape[0], size=3)
y_p

array([4, 4, 1])

In [ ]:
train_x[0].shape

(2,)